## Downloading NOAA Data for Raleigh, NC

Inspired by https://www.kaggle.com/johnjdavisiv/us-counties-weather-health-hospitals-covid19-data

Collected based on https://www.ncei.noaa.gov/access/search/data-search/global-summary-of-the-day?stations=72306013722

In [27]:
import pandas as pd
import numpy as np

In [28]:
# download data from NOAA for 2000-present
years = []

for y in range(2000, 2022):
    years.append(pd.read_csv("https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/{}/72306013722.csv".format(y)))

weather = pd.concat(years, ignore_index=True) # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [29]:
# add columns for day, month, year, and season

# split date
dates = weather['DATE'].str.split('-', expand=True) # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html
weather['YEAR'] = pd.to_numeric(dates[0]) # https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html#pandas.to_numeric
weather['MONTH'] = pd.to_numeric(dates[1])
weather['DAY'] = pd.to_numeric(dates[2])

In [30]:
weather.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR,MONTH,DAY
0,72306013722,2000-01-01,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",47.6,24,38.1,24,...,,33.1,,0.00,G,999.9,100000,2000,1,1
1,72306013722,2000-01-02,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",55.3,24,46.3,24,...,,33.1,,0.00,G,999.9,0,2000,1,2
2,72306013722,2000-01-03,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",62.6,24,55.4,24,...,,43.0,,0.00,G,999.9,100000,2000,1,3
3,72306013722,2000-01-04,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",65.2,24,58.6,24,...,,55.0,,0.00,G,999.9,110000,2000,1,4
4,72306013722,2000-01-05,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",45.7,24,30.9,24,...,*,37.0,*,0.34,G,999.9,110000,2000,1,5


In [31]:
# seasonal data from https://www.timeanddate.com/calendar/seasons.html?year=2000
# with location set to Durham, NC (very close to the Raleigh-Durham Airport, where this weather station is located)

seasons = pd.read_csv('seasons.csv', index_col=['Year'])
# months in which an equinox/solstice occurs
months = np.array(seasons.columns, dtype='str').reshape((4,2))
months

array([['March Equinox', 'March Time'],
       ['June Solstice', 'June Time'],
       ['September Equinox', 'Sept Time'],
       ['December Solstice', 'Dec Time']], dtype='<U17')

In [32]:
# for each year, calculate start of each season
for row in months:
    day = row[0]
    time = row[1]
    # change time to just am or pm
    seasons[time] = seasons[time].str.split(' ', expand=True)[1]
    # turn date into numeric day - first day of new season
    seasons[day] = pd.to_numeric(seasons[day].str.split('-', expand=True)[0])

seasons.head()

,March Equinox,March Time,June Solstice,June Time,September Equinox,Sept Time,December Solstice,Dec Time
Year,,,,,,,,
2000,20,am,20,pm,22,pm,21,am
2001,20,am,21,am,22,pm,21,pm
2002,20,pm,21,am,23,am,21,pm
2003,20,pm,21,pm,23,am,22,am
2004,20,am,20,pm,22,pm,21,am


In [33]:
# https://stackoverflow.com/questions/49228596/pandas-case-when-default-in-pandas
# https://pandas.pydata.org/docs/reference/api/pandas.Series.isin.html
# https://pandas.pydata.org/docs/reference/api/pandas.Series.lt.html#pandas.Series.lt
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html


weather['SEASON'] = np.select(
    [
        weather['MONTH'].isin([1,  2]), # winter jan-feb
        weather['MONTH'].isin([3]) & weather['DAY'].lt(seasons['March Equinox'][weather['YEAR']].reset_index(drop=True)), # winter before mar equinox
        weather['MONTH'].isin([3]) & weather['DAY'].ge(seasons['March Equinox'][weather['YEAR']].reset_index(drop=True)), # spring after mar equinox
        weather['MONTH'].isin([4,  5]), # spring apr-may
        weather['MONTH'].isin([6]) & weather['DAY'].lt(seasons['June Solstice'][weather['YEAR']].reset_index(drop=True)), # spring before june solstice
        weather['MONTH'].isin([6]) & weather['DAY'].ge(seasons['June Solstice'][weather['YEAR']].reset_index(drop=True)), # summer after june solstice
        weather['MONTH'].isin([7,  8]), # summer jul-aug
        weather['MONTH'].isin([9]) & weather['DAY'].lt(seasons['September Equinox'][weather['YEAR']].reset_index(drop=True)), # summer before sept equinox
        weather['MONTH'].isin([9]) & weather['DAY'].ge(seasons['September Equinox'][weather['YEAR']].reset_index(drop=True)), # fall after setp equinox
        weather['MONTH'].isin([10, 11]), # fall oct-nov
        weather['MONTH'].isin([12]) & weather['DAY'].lt(seasons['December Solstice'][weather['YEAR']].reset_index(drop=True)), # fall before dec solstice
        weather['MONTH'].isin([12]) & weather['DAY'].ge(seasons['December Solstice'][weather['YEAR']].reset_index(drop=True)) # winter after dec solstice
        
    ],
    [
        0, # winter
        0, # winter
        1, # spring
        1, # spring
        1, # spring
        2, # summer
        2, # summer
        2, # summer
        3, # fall
        3, # fall
        3, # fall
        0 # winter
    ],
    default = 'ERROR'
)

# check that all dates have been coded as seasons
(weather['SEASON'] == 'ERROR').any()

False

In [34]:
weather.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,...,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,YEAR,MONTH,DAY,SEASON
0,72306013722,2000-01-01,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",47.6,24,38.1,24,...,33.1,,0.00,G,999.9,100000,2000,1,1,0
1,72306013722,2000-01-02,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",55.3,24,46.3,24,...,33.1,,0.00,G,999.9,0,2000,1,2,0
2,72306013722,2000-01-03,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",62.6,24,55.4,24,...,43.0,,0.00,G,999.9,100000,2000,1,3,0
3,72306013722,2000-01-04,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",65.2,24,58.6,24,...,55.0,,0.00,G,999.9,110000,2000,1,4,0
4,72306013722,2000-01-05,35.8923,-78.7819,126.8,"RALEIGH AIRPORT, NC US",45.7,24,30.9,24,...,37.0,*,0.34,G,999.9,110000,2000,1,5,0


In [35]:
vars_present = ["Snow Depth",
"Average Wind Speed",
"Average Dew Point",
"Maximum Temperature",
"Average Visibility",
"Precipitation",
"Average Station Pressure",
"Maximum Wind Gust",
"Average Temperature",
"Minimum Temperature",
"Average Sea Level Pressure",
"Maximum Sustained Wind Speed",
"Indicators"]

col_key = {'YEAR': "Year",
           'MONTH': "Month",
           'DAY': "Day",
           'SEASON': "Season",
           'TEMP': "Average Temperature",
           'DEWP': "Average Dew Point",
           'SLP': "Average Sea Level Pressure",
           'STP': "Average Station Pressure",
           'VISIB': "Average Visibility",
           'WDSP': "Average Wind Speed",
           'MXSPD': "Maximum Sustained Wind Speed",
           'GUST': "Maximum Wind Gust",
           'MAX': "Maximum Temperature",
           'MIN': "Minimum Temperature",
           'PRCP': "Precipitation",
           'SNDP': "Snow Depth"}

list(col_key.keys())

['YEAR',
 'MONTH',
 'DAY',
 'SEASON',
 'TEMP',
 'DEWP',
 'SLP',
 'STP',
 'VISIB',
 'WDSP',
 'MXSPD',
 'GUST',
 'MAX',
 'MIN',
 'PRCP',
 'SNDP']

In [36]:
# drop unused columns & convert to numeric
weather_small = weather[list(col_key.keys())]
weather_small.apply(pd.to_numeric) # https://stackoverflow.com/questions/34844711/convert-entire-pandas-dataframe-to-integers-in-pandas-0-17-0/34844867

,YEAR,MONTH,DAY,SEASON,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP
0,2000,1,1,0,47.6,38.1,1023.7,999.9,8.3,3.0,10.1,999.9,66.9,33.1,0.00,999.9
1,2000,1,2,0,55.3,46.3,1024.2,999.9,9.5,4.8,14.0,999.9,70.0,33.1,0.00,999.9
2,2000,1,3,0,62.6,55.4,1021.3,999.9,8.4,8.5,14.0,999.9,73.9,43.0,0.00,999.9
3,2000,1,4,0,65.2,58.6,1014.4,999.9,9.5,15.3,28.0,35.9,73.9,55.0,0.00,999.9
4,2000,1,5,0,45.7,30.9,1019.8,999.9,9.8,6.4,11.1,18.1,57.9,37.0,0.34,999.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,2021,5,3,1,71.4,63.0,1011.8,996.6,9.4,9.7,15.0,21.0,78.1,66.9,0.00,999.9
7794,2021,5,4,1,72.0,66.2,1010.7,995.3,9.5,9.2,14.0,24.1,87.1,64.0,0.20,999.9
7795,2021,5,5,1,71.7,64.0,1009.1,993.7,10.0,8.7,14.0,21.0,87.1,63.0,0.20,999.9
7796,2021,5,6,1,64.1,47.4,1015.3,999.6,10.0,5.0,9.9,21.0,84.9,55.0,0.00,999.9


In [58]:
wthr = weather_small.copy()

# in these two cols, 999.99 is missing, set to 0 instead
wthr.loc[wthr['GUST'] == 999.9, 'GUST'] = 0 
wthr.loc[wthr['SNDP'] == 999.9, 'SNDP'] = 0

# remove 19 days with missing SLP
wthr = wthr.loc[wthr['SLP'] != 9999.9]
# and 4 days with missing MXSPD
wthr = wthr.loc[wthr['MXSPD'] != 999.9]
# and 3 more days with missing PRCP
wthr = wthr.loc[wthr['PRCP'] != 99.99]

wthr.tail(10)

,YEAR,MONTH,DAY,SEASON,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP
7788,2021,4,28,1,72.5,56.3,1017.8,2.1,10.0,10.2,17.1,24.1,84.9,51.1,0.0,0.0
7789,2021,4,29,1,74.9,60.7,1013.0,997.7,10.0,12.2,22.0,35.0,86.0,61.0,0.0,0.0
7790,2021,4,30,1,74.7,50.8,1009.0,993.7,10.0,11.4,18.1,26.0,86.0,66.9,0.0,0.0
7791,2021,5,1,1,64.1,31.9,1016.5,1.0,10.0,5.0,9.9,17.1,79.0,50.0,0.0,0.0
7792,2021,5,2,1,64.6,46.7,1015.9,0.3,10.0,7.7,15.9,21.0,84.0,48.9,0.0,0.0
7793,2021,5,3,1,71.4,63.0,1011.8,996.6,9.4,9.7,15.0,21.0,78.1,66.9,0.0,0.0
7794,2021,5,4,1,72.0,66.2,1010.7,995.3,9.5,9.2,14.0,24.1,87.1,64.0,0.2,0.0
7795,2021,5,5,1,71.7,64.0,1009.1,993.7,10.0,8.7,14.0,21.0,87.1,63.0,0.2,0.0
7796,2021,5,6,1,64.1,47.4,1015.3,999.6,10.0,5.0,9.9,21.0,84.9,55.0,0.0,0.0
7797,2021,5,7,1,51.6,47.2,1013.8,998.1,10.0,0.7,2.9,0.0,57.9,46.9,0.0,0.0


In [59]:
wthr.shape

(7772, 16)

In [60]:
# save clean data
wthr.to_csv('weather.csv')

## References Consulted